In [0]:
df= (spark.read
.option("header", "true")
.option("inferSchema", "true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"))

In [0]:
df_n= (spark.read
.option("header", "true")
.option("inferSchema", "true").csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv"))

In [0]:
# creating del/Volumes/workspace/ecommerce/ecommerce_data/delta/part-00002-73d7b96e-4a48-4d03-9d55-2fdbe1d2f76c.c000.snappy.parquetta format files
df.write.format("delta").mode("overwrite").save("/Volumes/workspace/ecommerce/ecommerce_data/delta")

In [0]:
df.createOrReplaceTempView("temp_df")
spark.sql("CREATE TABLE workspace.ecommerce.df_oct_delta USING DELTA AS SELECT * FROM temp_df")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("SELECT * FROM workspace.ecommerce.df_oct_delta LIMIT 10").show()

+-------------------+----------+----------+-------------------+--------------------+-----------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|      brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-----------+------+---------+--------------------+
|2019-10-02 14:26:48|      view|  23700090|2053013561847841545|furniture.bedroom...|       arua| 61.01|516348315|6e2193d0-fcb5-44f...|
|2019-10-02 14:26:48|      view|   1005008|2053013555631882655|electronics.smart...|     xiaomi| 99.75|542457608|9a9fdd60-02a6-4b8...|
|2019-10-02 14:26:49|      view|   4801033|2053013554658804075|electronics.audio...|plantronics| 12.69|513077305|59f64eb8-57ad-417...|
|2019-10-02 14:26:49|      view|   5000634|2053013566100866035|appliances.sewing...|     janome| 195.6|556043841|cf674fa8-55aa-4f4...|
|2019-10-02 14:26:49|      view|  14700162|205301355713

In [0]:
# Test schema enforcement
# trying to append df with wrong schema to the delta file and capturing the exception
try:
    df_wrong  = spark.createDataFrame([("a","b","c")], ["x","y","z"])
    df_wrong .write.format("delta").mode("append").save("/Volumes/workspace/ecommerce/ecommerce_data/delta")
except Exception as e:
    print(f"Schema enforcement: {e}")

Schema enforcement: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: de4a1670-f0ed-437c-ba74-d6c80964a817).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- event_time: timestamp (nullable = true)
-- event_type: string (nullable = true)
-- product_id: integer (nullable = true)
-- category_id: long (nullable = true)
-- category_code: string (nullable = true)
-- brand: string (nullable = true)
-- price: double (nullable = true)
-- user_id: integer (nullable = true)
-- user_session: string (nullable = true)


Data schema:
root
-- x: string (nullable = true)
-- y: string (nullable = true)
-- z: string (nullable = true)

         
Table ACLs are enabled in this cluster, so automati

In [0]:
# Handle duplicate inserts
# Target → existing Delta table (/delta/sales)
# Source → new incoming DataFrame (df)
from delta.tables import DeltaTable
delta_table = DeltaTable.forPath(spark, "/Volumes/workspace/ecommerce/ecommerce_data/delta")
delta_table.alias("target").merge(
    df_n.alias("source"),
    "target.user_session = source.user_session AND target.event_time = source.event_time"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
delta_table.toDF().count()

109950743